# Working with DataFrames.jl beyond CSV files

# Part 1: review of common storage formats

## Bogumił Kamiński
### June 25, 2023

What is covered in part 1:
* reading Stata files (also SAS, SPSS)
* reading R files
* reanding and writing SQLite database
* reanding and writing DuckDB database
* reanding and writing Parquet files

As a particular topic I will discuss handling of metadata by these storage formats.

For more details on working with metadata check my blog:
* https://bkamins.github.io/julialang/2022/12/02/metadata.html
* https://bkamins.github.io/julialang/2023/01/13/tmt.html

## Setup

In [1]:
]st

Status `C:\WORK\dev\DataFramesTutorials\JuliaCon2023-Tutorial\Project.toml`
  [ba30903b] CodecXz v0.7.0
  [a93c6f00] DataFrames v1.6.1
  [d2f5444f] DuckDB v0.8.1
  [7073ff75] IJulia v1.24.2
  [98572fba] Parquet2 v0.2.17
  [df47a6cb] RData v1.0.0
  [52522f7a] ReadStatTables v0.2.5
  [0aa819cd] SQLite v1.6.0
  [2913bbd2] StatsBase v0.34.0


In [2]:
using CodecXz

In [3]:
using DataFrames

In [4]:
using DuckDB

In [5]:
using Parquet2

In [6]:
using RData

In [7]:
using ReadStatTables

In [8]:
using SQLite

## Get the data from Stata

I fetched the file we work with from https://data.mendeley.com/datasets/zcvgc6m58h/1

ReadStatTables.jl supports Stata, SAS, and SPSS files. In this tutorial we will work with Stata file.

In [9]:
raw_wdi = ReadStatTables.readstat("WDI2009.dta")

11123×872 ReadStatTable:
   Row │  country   wbcode   year  AG_AGR_TRAC_NO  AG_CON_FERT_MT  AG_CON_FERT ⋯
       │   String  String3  Int16        Float64?        Float64?        Float ⋯
───────┼────────────────────────────────────────────────────────────────────────
     1 │    Aruba      ABW   1960         missing         missing         miss ⋯
     2 │    Aruba      ABW   1961         missing         missing         miss ⋯
     3 │    Aruba      ABW   1962         missing         missing         miss ⋯
     4 │    Aruba      ABW   1963         missing         missing         miss ⋯
     5 │    Aruba      ABW   1964         missing         missing         miss ⋯
     6 │    Aruba      ABW   1965         missing         missing         miss ⋯
     7 │    Aruba      ABW   1966         missing         missing         miss ⋯
     8 │    Aruba      ABW   1967         missing         missing         miss ⋯
     9 │    Aruba      ABW   1968         missing         missing         miss ⋯
   

In [10]:
typeof(raw_wdi)

ReadStatTable{ChainedReadStatColumns}

`ReadStatTable` is a Tables.jl table that is a custom type provided by ReadStatTables.jl.

We can work with it directly. As an example, let us investigate its metadata.

In [11]:
metadata(raw_wdi)

ReadStatMeta:
  row count           => 11123
  var count           => 872
  modified time       => 2010-01-08T11:17:00
  file format version => 114
  notes               => ["2", "dataset coded for stata as in Catini, Panizza and Saade. Macro Data 4 Stata", "December 2009"]
  file extension      => .dta

In [12]:
colmetadata(raw_wdi)

ColMetaIterator{ReadStatColMeta} with 872 entries:
  :country           => ReadStatColMeta(CountryName, %44s)
  :wbcode            => ReadStatColMeta(%9s)
  :year              => ReadStatColMeta(%9.0g)
  :AG_AGR_TRAC_NO    => ReadStatColMeta(Agricultural machinery, tractors, %10.0…
  :AG_CON_FERT_MT    => ReadStatColMeta(Fertilizer consumption (metric tons), %…
  :AG_CON_FERT_ZS    => ReadStatColMeta(Fertilizer consumption (100 grams per h…
  :AG_LND_AGRI_K2    => ReadStatColMeta(Agricultural land (sq. km), %10.0g)
  :AG_LND_AGRI_ZS    => ReadStatColMeta(Agricultural land (% of land area), %10…
  :AG_LND_ARBL_HA    => ReadStatColMeta(Arable land (hectares), %10.0g)
  :AG_LND_ARBL_HA_PC => ReadStatColMeta(Arable land (hectares per person), %10.…
  :AG_LND_ARBL_ZS    => ReadStatColMeta(Arable land (% of land area), %10.0g)
  :AG_LND_CREL_HA    => ReadStatColMeta(Land under cereal production (hectares)…
  :AG_LND_CROP_ZS    => ReadStatColMeta(Permanent cropland (% of land area), %1…
  :AG

In [13]:
colmetadata(raw_wdi, :AG_AGR_TRAC_NO)

ReadStatColMeta:
  label         => Agricultural machinery, tractors
  format        => %10.0g
  type          => READSTAT_TYPE_DOUBLE
  value label   => 
  storage width => 8
  display width => 10
  measure       => READSTAT_MEASURE_UNKNOWN
  alignment     => READSTAT_ALIGNMENT_RIGHT

I usually transform Tables.jl tables into `DataFrame` objects and later use functions from DataFrames.jl to work with them.

In [14]:
wdi = DataFrame(raw_wdi)

Row,country,wbcode,year,AG_AGR_TRAC_NO,AG_CON_FERT_MT,AG_CON_FERT_ZS,AG_LND_AGRI_K2,AG_LND_AGRI_ZS,AG_LND_ARBL_HA,AG_LND_ARBL_HA_PC,AG_LND_ARBL_ZS,AG_LND_CREL_HA,AG_LND_CROP_ZS,AG_LND_FRST_K2,AG_LND_FRST_ZS,AG_LND_IRIG_ZS,AG_LND_TOTL_K2,AG_LND_TRAC_ZS,AG_PRD_CROP_XD,AG_PRD_FOOD_XD,AG_PRD_LVSK_XD,AG_SRF_TOTL_K2,AG_YLD_CREL_KG,BG_GSR_NFSV_GD_ZS,BM_GSR_FCTY_CD,BM_GSR_GNFS_CD,BM_GSR_MRCH_CD,BM_GSR_NFSV_CD,BM_GSR_ROYL_CD,BM_GSR_TOTL_CD,BM_KLT_DINV_GD_ZS,BM_TRF_PWKR_CD_DT,BN_CAB_XOKA_CD,BN_CAB_XOKA_GD_ZS,BN_GSR_FCTY_CD,BN_GSR_GNFS_CD,BN_GSR_MRCH_CD,BN_KAC_EOMS_CD,BN_KLT_DINV_CD,BN_KLT_PTXL_CD,BN_RES_INCL_CD,BN_TRF_CURR_CD,BN_TRF_KOGT_CD,BX_GSR_CCIS_CD,BX_GSR_CCIS_ZS,BX_GSR_FCTY_CD,BX_GSR_GNFS_CD,BX_GSR_MRCH_CD,BX_GSR_NFSV_CD,BX_GSR_ROYL_CD,BX_GSR_TOTL_CD,BX_KLT_DINV_CD_WD,BX_KLT_DINV_WD_GD_ZS,BX_PEF_TOTL_CD_DT,BX_TRF_CURR_CD,BX_TRF_PWKR_CD,BX_TRF_PWKR_CD_DT,BX_TRF_PWKR_DT_GD_ZS,CM_FIN_INTL_GD_ZS,CM_MKT_INDX_ZG,CM_MKT_LCAP_CD,CM_MKT_LCAP_GD_ZS,CM_MKT_LDOM_NO,CM_MKT_TRAD_CD,CM_MKT_TRAD_GD_ZS,CM_MKT_TRNR,DC_DAC_AUSL_CD,DC_DAC_AUTL_CD,DC_DAC_BELL_CD,DC_DAC_CANL_CD,DC_DAC_CECL_CD,DC_DAC_CHEL_CD,DC_DAC_DEUL_CD,DC_DAC_DNKL_CD,DC_DAC_ESPL_CD,DC_DAC_FINL_CD,DC_DAC_FRAL_CD,DC_DAC_GBRL_CD,DC_DAC_GRCL_CD,DC_DAC_IRLL_CD,DC_DAC_ITAL_CD,DC_DAC_JPNL_CD,DC_DAC_LUXL_CD,DC_DAC_NLDL_CD,DC_DAC_NORL_CD,DC_DAC_NZLL_CD,DC_DAC_PRTL_CD,DC_DAC_SWEL_CD,DC_DAC_TOTL_CD,DC_DAC_USAL_CD,DT_DOD_DECT_CD,DT_DOD_DECT_GN_ZS,DT_DOD_DIMF_CD,DT_DOD_DLXF_CD,DT_DOD_DPNG_CD,DT_DOD_DPPG_CD,DT_DOD_DSTC_CD,DT_DOD_DSTC_IR_ZS,DT_DOD_DSTC_XP_ZS,DT_DOD_DSTC_ZS,⋯
,String,String3,Int16,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,⋯
1,Aruba,ABW,1960,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,⋯
2,Aruba,ABW,1961,missing,missing,missing,20.0,11.1111,2000.0,0.0398057,11.1111,missing,missing,missing,missing,missing,180.0,missing,missing,missing,missing,180.0,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missi

Note that the metadata is retained in `DataFrame`. The diffrence is that it is exposed as standard `Dict` objects instead of custom objects offered by ReadStatTables.jl.

In [15]:
metadata(wdi)

Dict{String, Any} with 13 entries:
  "file_ext"             => ".dta"
  "modified_time"        => DateTime("2010-01-08T11:17:00")
  "file_format_version"  => 114
  "file_format_is_64bit" => false
  "table_name"           => ""
  "notes"                => ["2", "dataset coded for stata as in Catini, Panizz…
  "file_encoding"        => ""
  "file_label"           => ""
  "var_count"            => 872
  "row_count"            => 11123
  "creation_time"        => DateTime("2010-01-08T11:17:00")
  "endianness"           => READSTAT_ENDIAN_LITTLE
  "compression"          => READSTAT_COMPRESS_NONE

In [16]:
colmetadata(wdi)

Dict{Symbol, Dict{String, Any}} with 872 entries:
  :GC_XPN_OTHR_ZS       => Dict("label"=>"Other expense (% of expense)", "forma…
  :SH_DYN_AIDS_ZS       => Dict("label"=>"Prevalence of HIV, total (% of popula…
  :EP_PMP_DESL_CD       => Dict("label"=>"Pump price for diesel fuel (US\$ per …
  :SL_TLF_PRIM_ZS       => Dict("label"=>"Labor force with primary education (%…
  :NE_CON_PRVT_PC_KD_ZG => Dict("label"=>"Household final consumption expenditu…
  :PA_NUS_PPP           => Dict("label"=>"PPP conversion factor, GDP (LCU per i…
  :DT_NFL_WFPG_CD       => Dict("label"=>"UN net multilateral flows, WFP (curre…
  :AG_LND_CREL_HA       => Dict("label"=>"Land under cereal production (hectare…
  :EE_BOD_TOTL_KG       => Dict("label"=>"Organic water pollutant (BOD) emissio…
  :SH_DYN_CHLD_MA       => Dict("label"=>"Mortality rate, male child (per 1,000…
  :ER_LND_PTLD_ZS       => Dict("label"=>"Nationally protected areas (% of tota…
  :IP_JRN_ARTC_SC       => Dict("label"=>"Scientific and te

In [17]:
colmetadata(wdi, :AG_AGR_TRAC_NO)

Dict{String, Any} with 8 entries:
  "label"         => "Agricultural machinery, tractors"
  "format"        => "%10.0g"
  "display_width" => 10
  "measure"       => READSTAT_MEASURE_UNKNOWN
  "alignment"     => READSTAT_ALIGNMENT_RIGHT
  "type"          => READSTAT_TYPE_DOUBLE
  "storage_width" => 0x0000000000000008
  "vallabel"      => Symbol("")

# Get data from R

I created the file we used based on standard `cars` data set available in R.

In [18]:
r_objs = load("cars.RData")

Dict{String, Any} with 1 entry:
  "cars" => 50×2 DataFrame…

Note that `.RData` files can store multiple objects, so we need to extract the `cars` data frame from `r_objs`:

In [19]:
cars = r_objs["cars"]

Row,speed,dist
,Float64,Float64
1,4.0,2.0
2,4.0,10.0
3,7.0,4.0
4,7.0,22.0
5,8.0,16.0
6,9.0,10.0
7,10.0,18.0
8,10.0,26.0
9,10.0,34.0


Loading CodecXz.jl package was required to be loaded for decompression (you can check that loading would fail if we had not done `using CodecXz` in setup).

Observe that RData.jl does not expose a custom table type, but instead provides a DataFrames.jl `DataFrame`.

Since we can attach attributes to objects in R, we can recover these attributes in Julia as metadata:

In [20]:
metadata(cars)

Dict{String, Any} with 3 entries:
  "source"    => "Ezekiel, M. (1930) Methods of Correlation Analysis. Wiley."
  "row.names" => Union{Missing, Int32}[missing, -50]
  "comment"   => "The data give the speed of cars and the distances taken to st…

In [21]:
colmetadata(cars)

Dict{Symbol, Dict{String, String}} with 2 entries:
  :dist  => Dict("label"=>"Stopping distance (ft)")
  :speed => Dict("label"=>"Speed (mph)")

## Write and read SQLite

We can create an in memory SQLite data base, but in this tutorial we explore options for storing our data on disk.

First clean up the working directory.

In [22]:
isfile("cars.sqlite") && rm("cars.sqlite")

false

Create a new database:

In [23]:
cars_sqlite = SQLite.DB("cars.sqlite")

SQLite.DB("cars.sqlite")

Check that it does not contain any tables yet:

In [24]:
SQLite.tables(cars_sqlite)

SQLite.DBTable[]

Use the `load!` function to load the `cars` data frame to SQLite table.

In [25]:
SQLite.load!(cars, cars_sqlite, "cars")

"cars"

Check that indeed a new table has been created:

In [26]:
SQLite.tables(cars_sqlite)

1-element Vector{SQLite.DBTable}:
 SQLite.DBTable("cars", Tables.Schema:
 :speed  Union{Missing, Float64}
 :dist   Union{Missing, Float64})

Execute some query on this table and fetch the result as a `DataFrame`:

In [27]:
DBInterface.execute(cars_sqlite, "SELECT * FROM cars WHERE speed < 9.5") |> DataFrame

Row,speed,dist
,Float64,Float64
1,4.0,2.0
2,4.0,10.0
3,7.0,4.0
4,7.0,22.0
5,8.0,16.0
6,9.0,10.0


Create a new table within SQLite:

In [28]:
DBInterface.execute(cars_sqlite, "CREATE TABLE slow_cars AS SELECT * FROM cars WHERE speed < 9.5");

In [29]:
SQLite.tables(cars_sqlite)

2-element Vector{SQLite.DBTable}:
 SQLite.DBTable("cars", Tables.Schema:
 :speed  Union{Missing, Float64}
 :dist   Union{Missing, Float64})
 SQLite.DBTable("slow_cars", Tables.Schema:
 :speed  Union{Missing, Float64}
 :dist   Union{Missing, Float64})

Close the connection to the data base:

In [30]:
DBInterface.close!(cars_sqlite)

Reopen it to read data:

In [31]:
cars_sqlite = SQLite.DB("cars.sqlite")

SQLite.DB("cars.sqlite")

In [32]:
SQLite.tables(cars_sqlite)

2-element Vector{SQLite.DBTable}:
 SQLite.DBTable("cars", Tables.Schema:
 :speed  Union{Missing, Float64}
 :dist   Union{Missing, Float64})
 SQLite.DBTable("slow_cars", Tables.Schema:
 :speed  Union{Missing, Float64}
 :dist   Union{Missing, Float64})

In [33]:
DBInterface.execute(cars_sqlite, "SELECT * FROM slow_cars") |> DataFrame

Row,speed,dist
,Float64,Float64
1,4.0,2.0
2,4.0,10.0
3,7.0,4.0
4,7.0,22.0
5,8.0,16.0
6,9.0,10.0


In [34]:
DBInterface.close!(cars_sqlite)

## Write and read DuckDB

Working with DuckDB is quite similar to SQLite. Let us focus on the differences.

In [35]:
isfile("cars.duckdb") && rm("cars.duckdb")

false

In [36]:
cars_duckdb = DBInterface.connect(DuckDB.DB, "cars.duckdb")

DuckDB.DB("cars.duckdb")

The `register_data_frame` function allows us to transfer the `cars` data frame to a data base:

In [37]:
DuckDB.register_data_frame(cars_duckdb, cars, "cars")

List tables as a data frame:

In [38]:
DBInterface.execute(cars_duckdb, "SHOW ALL TABLES") |> DataFrame

Row,database,schema,name,column_names,column_types,temporary
,String?,String?,String?,Array…?,Array…?,Bool?
1,cars,main,cars,"Union{Missing, String}[""speed"", ""dist""]","Union{Missing, String}[""DOUBLE"", ""DOUBLE""]",false


In [39]:
DBInterface.execute(cars_duckdb, "SELECT * FROM cars WHERE speed < 9.5") |> DataFrame

Row,speed,dist
,Float64?,Float64?
1,4.0,2.0
2,4.0,10.0
3,7.0,4.0
4,7.0,22.0
5,8.0,16.0
6,9.0,10.0


In [40]:
DBInterface.execute(cars_duckdb, "CREATE TABLE slow_cars AS SELECT * FROM cars WHERE speed < 9.5");

In [41]:
DBInterface.execute(cars_duckdb, "SHOW ALL TABLES") |> DataFrame

Row,database,schema,name,column_names,column_types,temporary
,String?,String?,String?,Array…?,Array…?,Bool?
1,cars,main,cars,"Union{Missing, String}[""speed"", ""dist""]","Union{Missing, String}[""DOUBLE"", ""DOUBLE""]",false
2,cars,main,slow_cars,"Union{Missing, String}[""speed"", ""dist""]","Union{Missing, String}[""DOUBLE"", ""DOUBLE""]",false


In [42]:
DBInterface.execute(cars_duckdb, "SELECT * FROM slow_cars") |> DataFrame

Row,speed,dist
,Float64?,Float64?
1,4.0,2.0
2,4.0,10.0
3,7.0,4.0
4,7.0,22.0
5,8.0,16.0
6,9.0,10.0


In [43]:
DBInterface.close!(cars_duckdb)

Notes:
1. `close!` does not free file handle correctly currently. This is going to be fixed soon.
2. In upcoming releases of DuckDB.jl it will get an improved integration with Tables.jl tables (current release is focused on integration with DataFrames.jl).

## Write and read Parquet

We store our data in a folder, as Parquet allows for multi-file data stores.

In [44]:
isdir("pq_data") && rm("pq_data"; recursive=true)

false

In [45]:
mkdir("pq_data")

"pq_data"

A basic workflow:

In [46]:
Parquet2.writefile("pq_data/cars.parquet", cars)

✏ Parquet2.FileWriter{IOStream}(pq_data/cars.parquet)

In [47]:
Parquet2.readfile("pq_data/cars.parquet")

≔ Parquet2.Dataset (775 bytes)
	1. "speed": Float64
	2. "dist": Float64


Note that below we pass `copycols=false` to save memory (there is no risk of re-using the data; this is going to be explained in more detail in part 3 of the tutorial):

In [48]:
cars2 = DataFrame(Parquet2.readfile("pq_data/cars.parquet"), copycols=false)

Row,speed,dist
,Float64,Float64
1,4.0,2.0
2,4.0,10.0
3,7.0,4.0
4,7.0,22.0
5,8.0,16.0
6,9.0,10.0
7,10.0,18.0
8,10.0,26.0
9,10.0,34.0


Parquet format stores metadata. Let us check that indedd it was retained:

In [49]:
metadata(cars2)

Dict{String, String} with 3 entries:
  "source"    => "Ezekiel, M. (1930) Methods of Correlation Analysis. Wiley."
  "row.names" => "Union{Missing, Int32}[missing, -50]"
  "comment"   => "The data give the speed of cars and the distances taken to st…

In [50]:
colmetadata(cars2)

Dict{Symbol, Dict{String, String}} with 2 entries:
  :dist  => Dict("label"=>"Stopping distance (ft)")
  :speed => Dict("label"=>"Speed (mph)")

Parquet file format allows us to split our data frame into several smaller row groups stored in one file.

In part 3 we will discuss when it is especially useful.

Here let us concentrate on learning how to do it:

In [51]:
cars_2part = Iterators.partition(cars, 25)

Base.Iterators.PartitionIterator{DataFrame}(50×2 DataFrame
 Row │ speed    dist    
     │ Float64  Float64 
─────┼──────────────────
   1 │     4.0      2.0
   2 │     4.0     10.0
   3 │     7.0      4.0
   4 │     7.0     22.0
   5 │     8.0     16.0
   6 │     9.0     10.0
   7 │    10.0     18.0
   8 │    10.0     26.0
   9 │    10.0     34.0
  10 │    11.0     17.0
  11 │    11.0     28.0
  ⋮  │    ⋮        ⋮
  41 │    20.0     52.0
  42 │    20.0     56.0
  43 │    20.0     64.0
  44 │    22.0     66.0
  45 │    23.0     54.0
  46 │    24.0     70.0
  47 │    24.0     92.0
  48 │    24.0     93.0
  49 │    24.0    120.0
  50 │    25.0     85.0
         29 rows omitted, 25)

In [52]:
collect(cars_2part)

2-element Vector{SubDataFrame{DataFrame, DataFrames.Index, UnitRange{Int64}}}:
 25×2 SubDataFrame
 Row │ speed    dist    
     │ Float64  Float64 
─────┼──────────────────
   1 │     4.0      2.0
   2 │     4.0     10.0
   3 │     7.0      4.0
   4 │     7.0     22.0
   5 │     8.0     16.0
   6 │     9.0     10.0
   7 │    10.0     18.0
   8 │    10.0     26.0
   9 │    10.0     34.0
  10 │    11.0     17.0
  11 │    11.0     28.0
  ⋮  │    ⋮        ⋮
  16 │    13.0     26.0
  17 │    13.0     34.0
  18 │    13.0     34.0
  19 │    13.0     46.0
  20 │    14.0     26.0
  21 │    14.0     36.0
  22 │    14.0     60.0
  23 │    14.0     80.0
  24 │    15.0     20.0
  25 │    15.0     26.0
          4 rows omitted
 25×2 SubDataFrame
 Row │ speed    dist    
     │ Float64  Float64 
─────┼──────────────────
   1 │    15.0     54.0
   2 │    16.0     32.0
   3 │    16.0     40.0
   4 │    17.0     32.0
   5 │    17.0     40.0
   6 │    17.0     50.0
   7 │    18.0     42.0
   8 │    18.0 

In [53]:
Parquet2.writefile("pq_data/cars2.parquet", cars_2part)

✏ Parquet2.FileWriter{IOStream}(pq_data/cars2.parquet)

In [54]:
cars2_pq = Parquet2.readfile("pq_data/cars2.parquet")

≔ Parquet2.Dataset (856 bytes)
	1. "speed": Float64
	2. "dist": Float64


If we pass the `cars2_pq` to the `DataFrame` constructor we get what we had previously:

In [55]:
DataFrame(cars2_pq, copycols=false)

Row,speed,dist
,Float64,Float64
1,4.0,2.0
2,4.0,10.0
3,7.0,4.0
4,7.0,22.0
5,8.0,16.0
6,9.0,10.0
7,10.0,18.0
8,10.0,26.0
9,10.0,34.0


However, actually it is a collection of smaller tables, called `RowGroup`. In our case we have 2 groups:

In [56]:
length(cars2_pq)

2

We can fetch them separately as `DataFrame`:

In [57]:
DataFrame.(cars2_pq)

2-element Vector{DataFrame}:
 25×2 DataFrame
 Row │ speed    dist    
     │ Float64  Float64 
─────┼──────────────────
   1 │     4.0      2.0
   2 │     4.0     10.0
   3 │     7.0      4.0
   4 │     7.0     22.0
   5 │     8.0     16.0
   6 │     9.0     10.0
   7 │    10.0     18.0
   8 │    10.0     26.0
   9 │    10.0     34.0
  10 │    11.0     17.0
  11 │    11.0     28.0
  ⋮  │    ⋮        ⋮
  16 │    13.0     26.0
  17 │    13.0     34.0
  18 │    13.0     34.0
  19 │    13.0     46.0
  20 │    14.0     26.0
  21 │    14.0     36.0
  22 │    14.0     60.0
  23 │    14.0     80.0
  24 │    15.0     20.0
  25 │    15.0     26.0
          4 rows omitted
 25×2 DataFrame
 Row │ speed    dist    
     │ Float64  Float64 
─────┼──────────────────
   1 │    15.0     54.0
   2 │    16.0     32.0
   3 │    16.0     40.0
   4 │    17.0     32.0
   5 │    17.0     40.0
   6 │    17.0     50.0
   7 │    18.0     42.0
   8 │    18.0     56.0
   9 │    18.0     76.0
  10 │    18.0     84.0

Note that in `pq_data` folder we now have two files:

In [58]:
readdir("pq_data")

2-element Vector{String}:
 "cars.parquet"
 "cars2.parquet"

Assuming all files stored in it have the same schema, we can view them as a larger `DataSet` (multi-file, and each file can potentially have multiple `RowGroup`s):

In [59]:
cars_multi = Parquet2.readfile("pq_data", load_initial=true)

≔ Parquet2.Dataset (775 bytes)
	1. "speed": Float64
	2. "dist": Float64


Again, just passing it to `DataFrame` collects all into one output table:

In [60]:
DataFrame(cars_multi, copycols=false)

Row,speed,dist
,Float64,Float64
1,4.0,2.0
2,4.0,10.0
3,7.0,4.0
4,7.0,22.0
5,8.0,16.0
6,9.0,10.0
7,10.0,18.0
8,10.0,26.0
9,10.0,34.0


However, we have three row groups (1 from the first file and 2 from the second file):

In [61]:
length(cars_multi)

3

In [62]:
foreach(println∘DataFrame, cars_multi)

50×2 DataFrame
 Row │ speed    dist    
     │ Float64  Float64 
─────┼──────────────────
   1 │     4.0      2.0
   2 │     4.0     10.0
   3 │     7.0      4.0
   4 │     7.0     22.0
   5 │     8.0     16.0
   6 │     9.0     10.0
   7 │    10.0     18.0
   8 │    10.0     26.0
   9 │    10.0     34.0
  10 │    11.0     17.0
  11 │    11.0     28.0
  12 │    12.0     14.0
  13 │    12.0     20.0
  14 │    12.0     24.0
  15 │    12.0     28.0
  16 │    13.0     26.0
  17 │    13.0     34.0
  18 │    13.0     34.0
  19 │    13.0     46.0
  20 │    14.0     26.0
  21 │    14.0     36.0
  22 │    14.0     60.0
  23 │    14.0     80.0
  24 │    15.0     20.0
  25 │    15.0     26.0
  26 │    15.0     54.0
  27 │    16.0     32.0
  28 │    16.0     40.0
  29 │    17.0     32.0
  30 │    17.0     40.0
  31 │    17.0     50.0
  32 │    18.0     42.0
  33 │    18.0     56.0
  34 │    18.0     76.0
  35 │    18.0     84.0
  36 │    19.0     36.0
  37 │    19.0     46.0
  38 │    19.0     68.

In [63]:
nrow.(cars_multi)

3-element Vector{Int64}:
 50
 25
 25

In part 2 we will focus on discussing how one can work with data larger than RAM using Parquet2.jl.

*Preparation of this worksop has been supported by the Polish National Agency for Academic Exchange under the Strategic Partnerships programme, grant number BPI/PST/2021/1/00069/U/00001.*

![SGH & NAWA](logo.png)